# Klasa TCM

In [1]:
from pymprog import begin, var, maximize, solve, end, iprod, par, minimize, vobj, solver, glpk, sensitivity
from time import clock
import random
from numpy import shape

class TCM(object):
    
    def __init__(self,name,t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb):
        '''t-tablica czasow trwania
        tco - sprzezenia czasowe pomiedzy obiektami
        tcb - sprzezenia czasowe pomiedzy brygadami
        wco - wagi sprzezen pomiedzy obiektami
        wcb - wagi sprzezn pomiedzy brygadami
        '''
        #const        
        self.ro=0.0001
        
        #param
        self.name=name
        
        self.t=t
        
        self.tcdo=tcdo
        self.tcdb=tcdb
        
        self.tcgo=tcgo
        self.tcgb=tcgb
        
        self.cwdo=cwdo
        self.cwdb=cwdb
        
        self.cwgo=cwgo
        self.cwgb=cwgb
        
        (self.onumber,self.bnumber)=shape(self.t)
        
        (self.Time,self.FC)=self.solve()

    def raport(self):
        print(self.name,":",self.Time," FC:",self.FC )
        
    def solve(self):
        time=clock()
        p=begin('TCM')
        W=iprod(range(self.onumber),range(self.bnumber)) # tablica zamiennych decyzyjnych dla NWR NWZ NPR NPZ
        O=iprod(range(self.onumber-1),range(self.bnumber)) #tablica zmiennych dla cko - sprzezen korygujacych pomiedzy obiektami
        B=iprod(range(self.onumber),range(self.bnumber-1)) #tablica zmiennych dla ckb - sprzezen korygujacych pomiedzy brygadami
        
        NWR=var('NWR',W) #czas trwania czynnosci
        NWZ=var('NWZ',W)
        NPR=var('NPR',W)
        NPZ=var('NPZ',W)
        ckdo=var('ckdo',O)
        ckdb=var('ckdb',B)
        ckgo=var('ckgo',O)
        ckgb=var('ckgb',B)
        ZC=var('ZC',W)
        
        t=par('t',self.t)
        
        tcdo=par('tcdo',self.tcdo) #parametry
        tcdb=par('tcdb',self.tcdb)
        
        tcgo=par('tcgo',self.tcgo)
        tcgb=par('tcgb',self.tcgb)
        
        cwdo=par('cwdo',self.cwdo)
        cwdb=par('cwdb',self.cwdb)
        
        cwgo=par('cwgo',self.cwgo)
        cwgb=par('cwgb',self.cwgb)
        

        #funkcja celu - minimalne koszty
        minimize(sum([ckdo[i,j]*cwdo[i][j]+ckgo[i,j]*cwgo[i][j] for (i,j) in O])+sum([ckdb[i,j]*cwdb[i][j]+ckgb[i,j]*cwgb[i][j] for (i,j) in B])+sum([NWZ[i,j]*self.ro-NPZ[i,j]*self.ro for (i,j) in W])+NWZ[self.onumber-1,self.bnumber-1])
        
        NWR[0,0] == 0
        
        for (i,j) in W:
            
            NWZ[i,j]==NWR[i,j]+t[i][j]
            NPZ[i,j]==NPR[i,j]+t[i][j]
            ZC[i,j]==NPZ[i,j]-NWZ[i,j]
            
        for (i,j) in O:
            NWR[i+1,j]>=NWZ[i,j]+tcdo[i][j]-ckdo[i,j]
            NPR[i+1,j]>=NPZ[i,j]+tcdo[i][j]-ckdo[i,j]
            
            NWR[i+1,j]<=NWZ[i,j]+tcgo[i][j]+ckgo[i,j]
            NPR[i+1,j]<=NPZ[i,j]+tcgo[i][j]+ckgo[i,j]
            
        for (i,j) in B:
            NWR[i,j+1]>=NWZ[i,j]+tcdb[i][j]-ckdb[i,j]
            NPR[i,j+1]>=NPZ[i,j]+tcdb[i][j]-ckdb[i,j]
            
            NWR[i,j+1]<=NWZ[i,j]+tcgb[i][j]+ckgb[i,j]
            NPR[i,j+1]<=NPZ[i,j]+tcgb[i][j]+ckgb[i,j]
            
        NWZ[self.onumber-1,self.bnumber-1]==NPZ[self.onumber-1,self.bnumber-1]
            
        
        #parametry solvera
        #solver(float,msg_lev=glpk.GLP_MSG_OFF)
        
        p.solve()
        
        print('FC:',vobj())
        
        for (i,j) in W:
            print('O',i+1,' B',j+1,'NWR',NWR[i,j].primal,'NWZ',NWZ[i,j].primal,'NPR',NPR[i,j].primal,'NPZ',NPZ[i,j].primal,'ZC',ZC[i,j].primal)
        for (i,j) in O:
            print(i+1,j+1,'ckdo',ckdo[i,j].primal,'ckgo',ckgo[i,j].primal)       
        for (i,j) in B:
            print(i+1,j+1,'ckdb',ckdb[i,j].primal,'ckgb',ckgb[i,j].primal)
        print('Termin realizacji:',NWZ[self.onumber-1,self.bnumber-1].primal)
        
        print('Czas kompilacji:',round(clock()-time,4),'s')
        
        return NWZ[self.onumber-1,self.bnumber-1].primal,vobj()
    
        #sensitivity()

# Model A.1: Brak ograniczeń

In [2]:
import random

t=[[random.randint(4,10) for i in range(4)] for j in range(3)]

t=[[7, 8, 6, 7], [9, 4, 7, 9], [10, 7, 7, 4]]

tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


A1=TCM('A.1',t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

FC: 43.99809999999998
O 1  B 1 NWR 0.0 NWZ 7.0 NPR 0.0 NPZ 7.0 ZC 0.0
O 1  B 2 NWR 7.0 NWZ 15.0 NPR 10.0 NPZ 18.0 ZC 3.0
O 1  B 3 NWR 15.0 NWZ 21.0 NPR 18.0 NPZ 24.0 ZC 3.0
O 1  B 4 NWR 21.0 NWZ 28.0 NPR 24.0 NPZ 31.0 ZC 3.0
O 2  B 1 NWR 7.0 NWZ 16.0 NPR 7.0 NPZ 16.0 ZC 0.0
O 2  B 2 NWR 16.0 NWZ 20.0 NPR 20.0 NPZ 24.0 ZC 4.0
O 2  B 3 NWR 21.0 NWZ 28.0 NPR 24.0 NPZ 31.0 ZC 3.0
O 2  B 4 NWR 28.0 NWZ 37.0 NPR 31.0 NPZ 40.0 ZC 3.0
O 3  B 1 NWR 16.0 NWZ 26.0 NPR 16.0 NPZ 26.0 ZC 0.0
O 3  B 2 NWR 26.0 NWZ 33.0 NPR 26.0 NPZ 33.0 ZC 0.0
O 3  B 3 NWR 33.0 NWZ 40.0 NPR 33.0 NPZ 40.0 ZC 0.0
O 3  B 4 NWR 40.0 NWZ 44.0 NPR 40.0 NPZ 44.0 ZC 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
2 1 ckdo 0.0 ckgo 0.0
2 2 ckdo 0.0 ckgo 0.0
2 3 ckdo 0.0 ckgo 0.0
2 4 ckdo 0.0 ckgo 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
1 3 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0
2 3 ckdb 0.0 ckgb 0.0
3 1 ckdb 0.0 ckgb 0.0
3 2 ckdb 0.0 ckgb 0.0
3 3 ckdb 

# Model A.2: Ciagłość pracy brygad

In [3]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[0,0,0,0],
    [0,0,0,0]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


A2=TCM("A.2",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

FC: 47.999999999999986
O 1  B 1 NWR 0.0 NWZ 7.0 NPR 0.0 NPZ 7.0 ZC 0.0
O 1  B 2 NWR 14.0 NWZ 22.0 NPR 14.0 NPZ 22.0 ZC 0.0
O 1  B 3 NWR 22.0 NWZ 28.0 NPR 22.0 NPZ 28.0 ZC 0.0
O 1  B 4 NWR 28.0 NWZ 35.0 NPR 28.0 NPZ 35.0 ZC 0.0
O 2  B 1 NWR 7.0 NWZ 16.0 NPR 7.0 NPZ 16.0 ZC 0.0
O 2  B 2 NWR 22.0 NWZ 26.0 NPR 22.0 NPZ 26.0 ZC 0.0
O 2  B 3 NWR 28.0 NWZ 35.0 NPR 28.0 NPZ 35.0 ZC 0.0
O 2  B 4 NWR 35.0 NWZ 44.0 NPR 35.0 NPZ 44.0 ZC 0.0
O 3  B 1 NWR 16.0 NWZ 26.0 NPR 16.0 NPZ 26.0 ZC 0.0
O 3  B 2 NWR 26.0 NWZ 33.0 NPR 26.0 NPZ 33.0 ZC 0.0
O 3  B 3 NWR 35.0 NWZ 42.0 NPR 35.0 NPZ 42.0 ZC 0.0
O 3  B 4 NWR 44.0 NWZ 48.0 NPR 44.0 NPZ 48.0 ZC 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
2 1 ckdo 0.0 ckgo 0.0
2 2 ckdo 0.0 ckgo 0.0
2 3 ckdo 0.0 ckgo 0.0
2 4 ckdo 0.0 ckgo 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
1 3 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0
2 3 ckdb 0.0 ckgb 0.0
3 1 ckdb 0.0 ckgb 0.0
3 2 ckdb 0.0 ckgb 0.0
3 3 ckd

# Model A.3: Ciagłość pracy na obiektach

In [4]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


A3=TCM("A.3",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

FC: 45.0
O 1  B 1 NWR 0.0 NWZ 7.0 NPR 0.0 NPZ 7.0 ZC 0.0
O 1  B 2 NWR 7.0 NWZ 15.0 NPR 7.0 NPZ 15.0 ZC 0.0
O 1  B 3 NWR 15.0 NWZ 21.0 NPR 15.0 NPZ 21.0 ZC 0.0
O 1  B 4 NWR 21.0 NWZ 28.0 NPR 21.0 NPZ 28.0 ZC 0.0
O 2  B 1 NWR 8.0 NWZ 17.0 NPR 8.0 NPZ 17.0 ZC 0.0
O 2  B 2 NWR 17.0 NWZ 21.0 NPR 17.0 NPZ 21.0 ZC 0.0
O 2  B 3 NWR 21.0 NWZ 28.0 NPR 21.0 NPZ 28.0 ZC 0.0
O 2  B 4 NWR 28.0 NWZ 37.0 NPR 28.0 NPZ 37.0 ZC 0.0
O 3  B 1 NWR 17.0 NWZ 27.0 NPR 17.0 NPZ 27.0 ZC 0.0
O 3  B 2 NWR 27.0 NWZ 34.0 NPR 27.0 NPZ 34.0 ZC 0.0
O 3  B 3 NWR 34.0 NWZ 41.0 NPR 34.0 NPZ 41.0 ZC 0.0
O 3  B 4 NWR 41.0 NWZ 45.0 NPR 41.0 NPZ 45.0 ZC 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
2 1 ckdo 0.0 ckgo 0.0
2 2 ckdo 0.0 ckgo 0.0
2 3 ckdo 0.0 ckgo 0.0
2 4 ckdo 0.0 ckgo 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
1 3 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0
2 3 ckdb 0.0 ckgb 0.0
3 1 ckdb 0.0 ckgb 0.0
3 2 ckdb 0.0 ckgb 0.0
3 3 ckdb 0.0 ckgb 0.0
T

# Model A.4: Ciagłość pracy dla 3 obiektu i 3 brygady

In [5]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,100,0,100],
    [100,100,0,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [0,0,0]]
    
cwdo=[[100,100,10000,100],
    [100,100,10000,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [10000,10000,10000]]
    
cwgo=[[100,100,10000,100],
    [100,100,10000,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [10000,10000,10000]]


A4=TCM("A.4",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

FC: 243.9989
O 1  B 1 NWR 0.0 NWZ 7.0 NPR 0.0 NPZ 7.0 ZC 0.0
O 1  B 2 NWR 7.0 NWZ 15.0 NPR 12.0 NPZ 20.0 ZC 5.0
O 1  B 3 NWR 20.0 NWZ 26.0 NPR 20.0 NPZ 26.0 ZC 0.0
O 1  B 4 NWR 26.0 NWZ 33.0 NPR 26.0 NPZ 33.0 ZC 0.0
O 2  B 1 NWR 7.0 NWZ 16.0 NPR 7.0 NPZ 16.0 ZC 0.0
O 2  B 2 NWR 16.0 NWZ 20.0 NPR 22.0 NPZ 26.0 ZC 6.0
O 2  B 3 NWR 26.0 NWZ 33.0 NPR 26.0 NPZ 33.0 ZC 0.0
O 2  B 4 NWR 33.0 NWZ 42.0 NPR 33.0 NPZ 42.0 ZC 0.0
O 3  B 1 NWR 16.0 NWZ 26.0 NPR 16.0 NPZ 26.0 ZC 0.0
O 3  B 2 NWR 26.0 NWZ 33.0 NPR 26.0 NPZ 33.0 ZC 0.0
O 3  B 3 NWR 33.0 NWZ 40.0 NPR 33.0 NPZ 40.0 ZC 0.0
O 3  B 4 NWR 40.0 NWZ 44.0 NPR 40.0 NPZ 44.0 ZC 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
2 1 ckdo 0.0 ckgo 0.0
2 2 ckdo 0.0 ckgo 0.0
2 3 ckdo 0.0 ckgo 0.0
2 4 ckdo 2.0 ckgo 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
1 3 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0
2 3 ckdb 0.0 ckgb 0.0
3 1 ckdb 0.0 ckgb 0.0
3 2 ckdb 0.0 ckgb 0.0
3 3 ckdb 0.0 ckgb 

# Model A.5: Jedna brygada może pracować na kilku obiektach

In [6]:
tcdo=[[-1,-1,-1,-1],
    [-1,-1,-1,-1]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


A5=TCM("A.5",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

FC: 41.99809999999999
O 1  B 1 NWR 0.0 NWZ 7.0 NPR 0.0 NPZ 7.0 ZC 0.0
O 1  B 2 NWR 7.0 NWZ 15.0 NPR 10.0 NPZ 18.0 ZC 3.0
O 1  B 3 NWR 15.0 NWZ 21.0 NPR 18.0 NPZ 24.0 ZC 3.0
O 1  B 4 NWR 21.0 NWZ 28.0 NPR 24.0 NPZ 31.0 ZC 3.0
O 2  B 1 NWR 6.0 NWZ 15.0 NPR 6.0 NPZ 15.0 ZC 0.0
O 2  B 2 NWR 15.0 NWZ 19.0 NPR 19.0 NPZ 23.0 ZC 4.0
O 2  B 3 NWR 20.0 NWZ 27.0 NPR 23.0 NPZ 30.0 ZC 3.0
O 2  B 4 NWR 27.0 NWZ 36.0 NPR 30.0 NPZ 39.0 ZC 3.0
O 3  B 1 NWR 14.0 NWZ 24.0 NPR 14.0 NPZ 24.0 ZC 0.0
O 3  B 2 NWR 24.0 NWZ 31.0 NPR 24.0 NPZ 31.0 ZC 0.0
O 3  B 3 NWR 31.0 NWZ 38.0 NPR 31.0 NPZ 38.0 ZC 0.0
O 3  B 4 NWR 38.0 NWZ 42.0 NPR 38.0 NPZ 42.0 ZC 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
2 1 ckdo 0.0 ckgo 0.0
2 2 ckdo 0.0 ckgo 0.0
2 3 ckdo 0.0 ckgo 0.0
2 4 ckdo 0.0 ckgo 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
1 3 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0
2 3 ckdb 0.0 ckgb 0.0
3 1 ckdb 0.0 ckgb 0.0
3 2 ckdb 0.0 ckgb 0.0
3 3 ckdb 

# Model A.6: Na jednym obiekcie może pracować więcej niż jedna brygada

In [7]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[-1,-1,-1],
    [-1,-1,-1],
    [-1,-1,-1]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


A6=TCM("A.6",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

FC: 40.998099999999994
O 1  B 1 NWR 0.0 NWZ 7.0 NPR 0.0 NPZ 7.0 ZC 0.0
O 1  B 2 NWR 6.0 NWZ 14.0 NPR 9.0 NPZ 17.0 ZC 3.0
O 1  B 3 NWR 13.0 NWZ 19.0 NPR 16.0 NPZ 22.0 ZC 3.0
O 1  B 4 NWR 18.0 NWZ 25.0 NPR 21.0 NPZ 28.0 ZC 3.0
O 2  B 1 NWR 7.0 NWZ 16.0 NPR 7.0 NPZ 16.0 ZC 0.0
O 2  B 2 NWR 15.0 NWZ 19.0 NPR 19.0 NPZ 23.0 ZC 4.0
O 2  B 3 NWR 19.0 NWZ 26.0 NPR 22.0 NPZ 29.0 ZC 3.0
O 2  B 4 NWR 25.0 NWZ 34.0 NPR 28.0 NPZ 37.0 ZC 3.0
O 3  B 1 NWR 16.0 NWZ 26.0 NPR 16.0 NPZ 26.0 ZC 0.0
O 3  B 2 NWR 25.0 NWZ 32.0 NPR 25.0 NPZ 32.0 ZC 0.0
O 3  B 3 NWR 31.0 NWZ 38.0 NPR 31.0 NPZ 38.0 ZC 0.0
O 3  B 4 NWR 37.0 NWZ 41.0 NPR 37.0 NPZ 41.0 ZC 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
2 1 ckdo 0.0 ckgo 0.0
2 2 ckdo 0.0 ckgo 0.0
2 3 ckdo 0.0 ckgo 0.0
2 4 ckdo 0.0 ckgo 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
1 3 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0
2 3 ckdb 0.0 ckgb 0.0
3 1 ckdb 0.0 ckgb 0.0
3 2 ckdb 0.0 ckgb 0.0
3 3 ckdb 

# Model A.7: Zazębianie się pomiędzy obiektami i brygadami

In [8]:
tcdo=[[-1,-1,-1,-1],
    [-1,-1,-1,-1]]
tcdb=[[-1,-1,-1],
    [-1,-1,-1],
    [-1,-1,-1]]
    
tcgo=[[100,100,100,100],
    [100,100,100,100]]
tcgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwdo=[[100,100,100,100],
    [100,100,100,100]]
cwdb=[[100,100,100],
    [100,100,100],
    [100,100,100]]
    
cwgo=[[100,100,100,100],
    [100,100,100,100]]
cwgb=[[100,100,100],
    [100,100,100],
    [100,100,100]]


A7=TCM("A.7",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

FC: 38.998099999999994
O 1  B 1 NWR 0.0 NWZ 7.0 NPR 0.0 NPZ 7.0 ZC 0.0
O 1  B 2 NWR 6.0 NWZ 14.0 NPR 9.0 NPZ 17.0 ZC 3.0
O 1  B 3 NWR 13.0 NWZ 19.0 NPR 16.0 NPZ 22.0 ZC 3.0
O 1  B 4 NWR 18.0 NWZ 25.0 NPR 21.0 NPZ 28.0 ZC 3.0
O 2  B 1 NWR 6.0 NWZ 15.0 NPR 6.0 NPZ 15.0 ZC 0.0
O 2  B 2 NWR 14.0 NWZ 18.0 NPR 18.0 NPZ 22.0 ZC 4.0
O 2  B 3 NWR 18.0 NWZ 25.0 NPR 21.0 NPZ 28.0 ZC 3.0
O 2  B 4 NWR 24.0 NWZ 33.0 NPR 27.0 NPZ 36.0 ZC 3.0
O 3  B 1 NWR 14.0 NWZ 24.0 NPR 14.0 NPZ 24.0 ZC 0.0
O 3  B 2 NWR 23.0 NWZ 30.0 NPR 23.0 NPZ 30.0 ZC 0.0
O 3  B 3 NWR 29.0 NWZ 36.0 NPR 29.0 NPZ 36.0 ZC 0.0
O 3  B 4 NWR 35.0 NWZ 39.0 NPR 35.0 NPZ 39.0 ZC 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
2 1 ckdo 0.0 ckgo 0.0
2 2 ckdo 0.0 ckgo 0.0
2 3 ckdo 0.0 ckgo 0.0
2 4 ckdo 0.0 ckgo 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
1 3 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0
2 3 ckdb 0.0 ckgb 0.0
3 1 ckdb 0.0 ckgb 0.0
3 2 ckdb 0.0 ckgb 0.0
3 3 ckdb 

# Model A.8a: Harmonogramowanie priorytetowe
Kolejność
B3
O2
B2

In [9]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,0,0,100],
    [100,0,0,100]]
tcgb=[[100,100,100],
    [0,0,0],
    [100,100,100]]
    
cwdo=[[100,10000,100000000,100],
    [100,10000,100000000,100]]
cwdb=[[100,100,100],
    [1000000,1000000,1000000],
    [100,100,100]]
    
cwgo=[[100,10000,100000000,100],
    [100,10000,100000000,100]]
cwgb=[[100,100,100],
    [1000000,1000000,1000000],
    [100,100,100]]


A8a=TCM("A.8a",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

FC: 840.0
O 1  B 1 NWR 0.0 NWZ 7.0 NPR 0.0 NPZ 7.0 ZC 0.0
O 1  B 2 NWR 8.0 NWZ 16.0 NPR 8.0 NPZ 16.0 ZC 0.0
O 1  B 3 NWR 14.0 NWZ 20.0 NPR 14.0 NPZ 20.0 ZC 0.0
O 1  B 4 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
O 2  B 1 NWR 7.0 NWZ 16.0 NPR 7.0 NPZ 16.0 ZC 0.0
O 2  B 2 NWR 16.0 NWZ 20.0 NPR 16.0 NPZ 20.0 ZC 0.0
O 2  B 3 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
O 2  B 4 NWR 27.0 NWZ 36.0 NPR 27.0 NPZ 36.0 ZC 0.0
O 3  B 1 NWR 10.0 NWZ 20.0 NPR 10.0 NPZ 20.0 ZC 0.0
O 3  B 2 NWR 20.0 NWZ 27.0 NPR 20.0 NPZ 27.0 ZC 0.0
O 3  B 3 NWR 27.0 NWZ 34.0 NPR 27.0 NPZ 34.0 ZC 0.0
O 3  B 4 NWR 36.0 NWZ 40.0 NPR 36.0 NPZ 40.0 ZC 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
2 1 ckdo 6.0 ckgo 0.0
2 2 ckdo 0.0 ckgo 0.0
2 3 ckdo 0.0 ckgo 0.0
2 4 ckdo 0.0 ckgo 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 2.0 ckgb 0.0
1 3 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 0.0
2 2 ckdb 0.0 ckgb 0.0
2 3 ckdb 0.0 ckgb 0.0
3 1 ckdb 0.0 ckgb 0.0
3 2 ckdb 0.0 ckgb 0.0
3 3 ckdb 0.0 ckgb 0.0


# Model A.8b
Kolejność
B3
O2
B2

In [10]:
tcdo=[[0,0,0,0],
    [0,0,0,0]]
tcdb=[[0,0,0],
    [0,0,0],
    [0,0,0]]
    
tcgo=[[100,0,0,100],
    [100,0,0,100]]
tcgb=[[100,100,100],
    [0,0,0],
    [100,100,100]]
    
cwdo=[[10000000000,10000,100000000,10000000000],
    [10000000000,10000,100000000,10000000000]]
cwdb=[[10000000000,10000000000,10000000000],
    [1000000,1000000,1000000],
    [10000000000,10000000000,10000000000]]
    
cwgo=[[100,10000,100000000,100],
    [100,10000,100000000,100]]
cwgb=[[100,100,100],
    [1000000,1000000,1000000],
    [100,100,100]]


A8b=TCM("A.8b",t,tcdo,tcdb,tcgo,tcgb,cwdo,cwdb,cwgo,cwgb)

FC: 6020046.0
O 1  B 1 NWR 0.0 NWZ 7.0 NPR 0.0 NPZ 7.0 ZC 0.0
O 1  B 2 NWR 12.0 NWZ 20.0 NPR 12.0 NPZ 20.0 ZC 0.0
O 1  B 3 NWR 20.0 NWZ 26.0 NPR 20.0 NPZ 26.0 ZC 0.0
O 1  B 4 NWR 26.0 NWZ 33.0 NPR 26.0 NPZ 33.0 ZC 0.0
O 2  B 1 NWR 7.0 NWZ 16.0 NPR 7.0 NPZ 16.0 ZC 0.0
O 2  B 2 NWR 20.0 NWZ 24.0 NPR 20.0 NPZ 24.0 ZC 0.0
O 2  B 3 NWR 26.0 NWZ 33.0 NPR 26.0 NPZ 33.0 ZC 0.0
O 2  B 4 NWR 33.0 NWZ 42.0 NPR 33.0 NPZ 42.0 ZC 0.0
O 3  B 1 NWR 16.0 NWZ 26.0 NPR 16.0 NPZ 26.0 ZC 0.0
O 3  B 2 NWR 26.0 NWZ 33.0 NPR 26.0 NPZ 33.0 ZC 0.0
O 3  B 3 NWR 33.0 NWZ 40.0 NPR 33.0 NPZ 40.0 ZC 0.0
O 3  B 4 NWR 42.0 NWZ 46.0 NPR 42.0 NPZ 46.0 ZC 0.0
1 1 ckdo 0.0 ckgo 0.0
1 2 ckdo 0.0 ckgo 0.0
1 3 ckdo 0.0 ckgo 0.0
1 4 ckdo 0.0 ckgo 0.0
2 1 ckdo 0.0 ckgo 0.0
2 2 ckdo 0.0 ckgo 2.0
2 3 ckdo 0.0 ckgo 0.0
2 4 ckdo 0.0 ckgo 0.0
1 1 ckdb 0.0 ckgb 0.0
1 2 ckdb 0.0 ckgb 0.0
1 3 ckdb 0.0 ckgb 0.0
2 1 ckdb 0.0 ckgb 4.0
2 2 ckdb 0.0 ckgb 2.0
2 3 ckdb 0.0 ckgb 0.0
3 1 ckdb 0.0 ckgb 0.0
3 2 ckdb 0.0 ckgb 0.0
3 3 ckdb 0.0 ckg

In [11]:
print('t:',t)
A=[A1,A2,A3,A4,A5,A6,A7,A8a,A8b]
for i in A:
    i.raport()

t: [[7, 8, 6, 7], [9, 4, 7, 9], [10, 7, 7, 4]]
A.1 : 44.0  FC: 43.99809999999998
A.2 : 48.0  FC: 47.999999999999986
A.3 : 45.0  FC: 45.0
A.4 : 44.0  FC: 243.9989
A.5 : 42.0  FC: 41.99809999999999
A.6 : 41.0  FC: 40.998099999999994
A.7 : 39.0  FC: 38.998099999999994
A.8a : 40.0  FC: 840.0
A.8b : 46.0  FC: 6020046.0
